In [ ]:
## try the drive .mount :https://stackoverflow.com/questions/69869534/files-and-folders-in-google-colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
import json
from tqdm.auto import tqdm

In [ ]:
start_file_path = '/content/drive/My Drive/articles/'

In [ ]:
df_articles = pd.read_pickle(start_file_path + "/df_articles.pkl")

In [ ]:
def get_tokenizer():
    return AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(start_file_path + '/model')
model = model.to(device="cuda")

In [ ]:
df_articles['source_domain'].value_counts()

,count
source_domain,
dailymail.co.uk,9188
independent.co.uk,9131
foxnews.com,9030
theguardian.com,8147
apnews.com,5953
newsweek.com,4860
hindustantimes.com,4587
nypost.com,4118
indianexpress.com,3761


In [ ]:
def relevant(label):
  if label == 'LABEL_1':
    return True
  return False

In [ ]:
df = df_articles[df_articles['source_domain'] == 'foxnews.com'].reset_index()

In [ ]:
tokenizer = get_tokenizer()

predicted_labels = []

for i in tqdm(range(len(df))):
  article = df.iloc[i]

  inputs = tokenizer(article['full_text'],  padding="max_length", truncation=True, max_length=512, return_tensors="pt")
  inputs = {key: value.to("cuda") for key, value in inputs.items()}

  with torch.no_grad():
    logits = model(**inputs).logits

  predicted_class_id = logits.argmax().item()
  predicted_labels.append(model.config.id2label[predicted_class_id])

df['predicted_label'] = predicted_labels
df['relevant'] = df['predicted_label'].map(relevant)

df_relevant = df[df['relevant'] == True].reset_index()
records = df_relevant.to_dict(orient='records')
with open(f'{start_file_path}/foxnews_relevant.json', 'w') as f:
    json.dump(records, f, indent=4)

  0%|          | 0/9030 [00:00<?, ?it/s]

In [ ]:
len(df_articles['source_domain'].unique())

15